# Gene statistic aggregation

In [1]:
import hail as hl
hl.init()

2021-11-09 15:13:33 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-11-09 15:13:34 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-6676655f87-9xllv:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/test_dataset/hail-20211109-1513-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [39]:
mt = hl.read_matrix_table('/home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/high_quality_variants_pao_removed.mt/')

In [38]:
mt.count()

(148267, 465)

In [40]:
mt = mt.annotate_rows(effect=mt.info.ANN.map(lambda x: x.split('\|')[1]),
                impact=mt.info.ANN.map(lambda x: x.split('\|')[2]),
                gene=mt.info.ANN.map(lambda x: x.split('\|')[3]))

mt = mt.annotate_rows(impact1=mt.impact[0].split('&'), effect1=mt.effect[0].split('&'), gene1=mt.gene[0].split('&'))

Count number of heterozygous genotypes per site.

In [43]:
mt = mt.annotate_rows(n_het=hl.agg.count_where(mt.GT.is_het()))

Group sites (rows) by gene name and collect stats:

* Number of high impact variants
* Number of sites
* Average heterozygosity

In [44]:
rows = mt.rows()
gene_stats = (rows.group_by(rows.gene1)
                       .aggregate(n_high_impact = hl.agg.count_where(rows.impact.contains('HIGH')),
                                  n_sites=hl.agg.count(),
                                  avg_het=hl.agg.mean(rows.n_het)))

Make a histogram of number of sites per gene.

In [8]:
p = hl.plot.histogram(gene_stats.n_sites, range=(0, 200), bins=100, title='Histogram of number of sites per gene')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 45:================================================>       (32 + 5) / 37]2021-11-09 15:19:42 Hail: INFO: Ordering unsorted dataset with network shuffle


In [14]:
p = hl.plot.histogram(gene_stats.avg_het, bins=100, title='Histogram of average gene heterozygosity')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 50:====================================================>   (35 + 2) / 37]2021-11-09 15:19:54 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 55:======================================================> (36 + 1) / 37]2021-11-09 15:20:03 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 58:======================================>                (26 + 11) / 37]

In [15]:
p = hl.plot.histogram(gene_stats.n_high_impact, bins=20, title='Histogram of number of high impact variants per gene')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 61:>                                                       (0 + 24) / 37]2021-11-09 15:20:07 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 65:===========================================>            (29 + 8) / 37]2021-11-09 15:20:10 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 68:===================================>                   (24 + 13) / 37]

Make a scatterplot of heterozygosity and impact per gene.

In [45]:
p = hl.plot.scatter(gene_stats.n_high_impact, gene_stats.avg_het, hover_fields={'Gene': gene_stats.gene1})

[Stage 78:======================================================> (36 + 1) / 37]2021-11-09 15:26:06 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 81:=====================================>                 (25 + 12) / 37]

In [46]:
p.xaxis.axis_label = '# high impact variants'
p.yaxis.axis_label = '# heterozygotes'
p.plot_width = 800
p.plot_height = 500
p.y_scale = LogScale()
p.x_scale = LogScale()
show(p)

In [48]:
high_impact_ht = gene_stats.filter(gene_stats.n_high_impact > 2)

In [50]:
high_impact_ht.to_pandas()

[Stage 88:=====================================>                 (25 + 12) / 37]2021-11-09 15:28:43 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 92:================>                                      (11 + 24) / 37]

,gene1,n_high_impact,n_sites,avg_het
0,[ABCA10],3,29,117.827586
1,[ABO],3,17,102.470588
2,[ASPN],6,8,76.250000
3,[C11orf40],3,6,89.500000
4,[C7orf55-LUC7L2],3,9,86.666667
5,[CAMKK2],5,18,64.611111
6,[CD207],3,14,97.357143
7,[CHI3L2],3,19,144.315789
8,[COL6A5],3,39,84.230769
9,[CYP2D6],3,3,22.333333
